<a href="https://colab.research.google.com/github/lipefp/python/blob/main/Projeto_novo_LP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

# 1. Carregar dados
df = pd.read_csv('Clientes_2024_1_1_convertido.csv')
df.columns = df.columns.str.strip()  # Remove espaços nos nomes das colunas

# 2. Preencher colunas simuladas
np.random.seed(42)
df['Renda'] = np.random.randint(1000, 6000, size=len(df))
df['Aposentado'] = np.where(df['Idade'] > 60, 1, 0)
df['Cartão de Crédito'] = np.random.choice([0, 1], size=len(df), p=[0.3, 0.7])
df['Já Pegou Empréstimo'] = np.random.choice([0, 1], size=len(df), p=[0.4, 0.6])
df['Empréstimo Atual'] = np.where(df['Já Pegou Empréstimo'] == 1, np.random.choice([0, 1], size=len(df), p=[0.7, 0.3]), 0)

# 3. Criar coluna alvo (Status = Aprovado ou não)
df['Status'] = np.where((df['Renda'] >= 3000) & (df['Aposentado'] == 0), 1, 0)

# 4. Treinar modelo SVM
X = df[['Idade', 'Renda', 'Aposentado', 'Cartão de Crédito', 'Já Pegou Empréstimo', 'Empréstimo Atual']]
y = df['Status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

modelo = make_pipeline(SimpleImputer(strategy='mean'), StandardScaler(), svm.SVC(gamma='auto'))
modelo.fit(X_train, y_train)

# 5. Classificar novo cliente
def classificar_cliente():
    print("\n--- Classificação de novo cliente ---")
    idade = int(input("Idade: "))
    renda = float(input("Renda: "))
    aposentado = int(input("Aposentado (0=Não, 1=Sim): "))
    cartao = int(input("Tem Cartão de Crédito (0=Não, 1=Sim): "))
    ja_pegou = int(input("Já Pegou Empréstimo (0=Não, 1=Sim): "))
    atual = int(input("Tem Empréstimo Atual (0=Não, 1=Sim): "))

    entrada = pd.DataFrame([{
        'Idade': idade,
        'Renda': renda,
        'Aposentado': aposentado,
        'Cartão de Crédito': cartao,
        'Já Pegou Empréstimo': ja_pegou,
        'Empréstimo Atual': atual
    }])

    resultado = modelo.predict(entrada)[0]
    print(f"\nResultado: {'Aprovado' if resultado == 1 else 'Reprovado'}")

# 6. Mostrar perfil de quem já pegou empréstimo
def mostrar_perfil_emp_ativo():
    base = df[df['Já Pegou Empréstimo'] == 1]

    print("\n===== PERFIL DE QUEM JÁ PEGOU EMPRÉSTIMO =====")

    print("\n📊 Idade média:")
    print(f"{base['Idade'].mean():.1f} anos")

    print("\n💰 Renda média:")
    print(f"R$ {base['Renda'].mean():.2f}")

    print("\n👴 Proporção de aposentados:")
    aposentados = base['Aposentado'].value_counts(normalize=True).to_dict()
    for k, v in aposentados.items():
        print(f"{'Sim' if k == 1 else 'Não'}: {v*100:.1f}%")

    print("\n🏘️ Bairros mais comuns:")
    if 'Bairro' in base.columns:
        print(base['Bairro'].value_counts())

    print("\n📦 Cartão de crédito:")
    cartao = base['Cartão de Crédito'].value_counts(normalize=True).to_dict()
    for k, v in cartao.items():
        print(f"{'Sim' if k == 1 else 'Não'}: {v*100:.1f}%")

    print("\n🔄 Empréstimo Atual (entre os que já pegaram):")
    emprestimos = base['Empréstimo Atual'].value_counts(normalize=True).to_dict()
    for k, v in emprestimos.items():
        print(f"{'Sim' if k == 1 else 'Não'}: {v*100:.1f}%")

    print("\n✅ Exemplo de perfil mais frequente:")
    print(f"Pessoa entre {int(base['Idade'].median() - 5)} e {int(base['Idade'].median() + 5)} anos,")
    print(f"com renda de aproximadamente R$ {base['Renda'].median():.2f},")
    print(f"{'aposentada' if base['Aposentado'].mode()[0] == 1 else 'não aposentada'},")
    print(f"e com cartão de crédito: {'Sim' if base['Cartão de Crédito'].mode()[0] == 1 else 'Não'}")

# 7. Menu interativo
while True:
    print("\n=== MENU ===")
    print("1 - Ver precisão do modelo")
    print("2 - Classificar novo cliente")
    print("3 - Ver perfil de quem já pegou empréstimo")
    print("0 - Sair")
    escolha = input("Escolha uma opção: ")

    if escolha == '1':
        acc = accuracy_score(y_test, modelo.predict(X_test))
        print(f"\nPrecisão do modelo: {acc:.2f}")
    elif escolha == '2':
        classificar_cliente()
    elif escolha == '3':
        mostrar_perfil_emp_ativo()
    elif escolha == '0':
        print("Encerrando.")
        break
    else:
        print("Opção inválida.")



=== MENU ===
1 - Ver precisão do modelo
2 - Classificar novo cliente
3 - Ver perfil de quem já pegou empréstimo
0 - Sair
Escolha uma opção: 1

Precisão do modelo: 1.00

=== MENU ===
1 - Ver precisão do modelo
2 - Classificar novo cliente
3 - Ver perfil de quem já pegou empréstimo
0 - Sair
Escolha uma opção: 3

===== PERFIL DE QUEM JÁ PEGOU EMPRÉSTIMO =====

📊 Idade média:
71.0 anos

💰 Renda média:
R$ 3488.65

👴 Proporção de aposentados:
Sim: 78.5%
Não: 21.5%

🏘️ Bairros mais comuns:
Bairro
Icaraí               9
Santa Rosa           5
Ingá                 5
São Francisco        5
Barreto              4
Itaipu               3
Bairro de Fátima     2
Ititioca             2
Jurujuba             2
Ponta d'Areia        2
Centro               2
Camboinhas           2
Largo da Batalha     2
Ilha da Conceição    2
Cubango              2
Charitas             2
Santa Bárbara        2
Engenho do Mato      2
Piratininga          2
São Domingos         1
Boa Viagem           1
Baldeador            